<a href="https://colab.research.google.com/github/Mastermind0309/ACP-in-Frail-Geriatric-Patients/blob/main/ACP_segments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from statsmodels.stats.contingency_tables import mcnemar
!pip install pandas statsmodels pingouin --quiet

import pandas as pd
import numpy as np
from scipy import stats
import statsmodels.api as sm
import pingouin as pg   # for McNemar test
from google.colab import drive
import re, unicodedata

drive.mount('/content/drive')

file_path = "/content/drive/MyDrive/ACP/ACP_csv1202.csv"

#  Very important: use Big5 encoding (Taiwan)
df = pd.read_csv(file_path, encoding="big5")

# Clean column names: remove leading/trailing spaces
df.columns = df.columns.str.strip()

print("===== HEAD =====")
print(df.head())
print("\n===== COLUMNS =====")
print(df.columns)

# Extract the year part (e.g. 73 from "73y8m")
df["年齡_yr"] = df["年齡"].astype(str).str.extract(r'(\d+)').astype(float)
print("\n===== 年齡 preview =====")
print(df[["年齡", "年齡_yr"]].head())

group_map = {
    "實驗組": 1,
    "對照組": 0,
    1: 1,
    2: 0
}
df["group01"] = df["組別"].map(group_map)

def score_knowledge(df, prefix):
    cols = [f"{prefix}_{i}" for i in range(1,7)]
    sub = df[cols]

    corr = pd.DataFrame(index=sub.index)
    corr[cols[0]] = (sub[cols[0]] == 1).astype(int)
    for c in cols[1:5]:
        corr[c] = (sub[c] == 2).astype(int)
    corr[cols[5]] = (sub[cols[5]] == 3).astype(int)

    return corr.sum(axis=1)

df["K1"] = score_knowledge(df, "壹_六_知識問卷")
df["K2"] = score_knowledge(df, "貳_三_知識問卷")
df["K3"] = score_knowledge(df, "參_三_知識問卷")
df["K_change"] = df["K3"] - df["K1"]
df["HADS_change"] = df["參_四_HADS_total"] - df["壹_七_HADS_total"]


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.4/204.4 kB 10.1 MB/s eta 0:00:00
Mounted at /content/drive
===== HEAD =====
   編號        收案日期 追蹤日期   組別  性別      年齡  教育程度  宗教信仰         職業  婚姻狀態  ...  \
0   1  2024/11/19  NaN  實驗組   1   73y8m     6     2     商、自營公司     2  ...   
1   2  2024/11/19  NaN  對照組   2  89y11m     5     3  商、銀行出納/退休     2  ...   
2   3  2024/12/19  NaN  實驗組   2  89y10m     3     2         家管     4  ...   
3   4  2024/12/19  NaN  實驗組   1   88y6m     5     2     技術員/退休     2  ...   
4   5   2025/2/20  NaN  對照組   2   73y6m     3     3         家管     2  ...   

   參_四_HADS_8  參_四_HADS_9  參_四_HADS_10  參_四_HADS_11  參_四_HADS_12  參_四_HADS_13  \
0         1.0         0.0          2.0          0.0          0.0          0.0   
1         0.0         0.0          3.0          1.0          3.0          0.0   
2         3.0         0.0          3.0          2.0          3.0          0.0   
3         0.0         0.0          0.0          0.0          0.0          1.0   
4       

In [2]:
def run_full_analysis():
    # ============ PRE/POST (continuous) ============
    for g in [0, 1]:
        print("\n========== Group", g, "(0=control,1=intervention) ==========\n")
        dfg = df[df["group01"] == g]

        print("Knowledge Pre vs Post (K1 vs K3):")
        print(stats.ttest_rel(dfg["K1"], dfg["K3"], nan_policy='omit'))

        print("\nHADS Pre vs Post:")
        print(stats.ttest_rel(dfg["壹_七_HADS_total"], dfg["參_四_HADS_total"], nan_policy='omit'))

    # ============ BETWEEN-GROUP continuous ============
    cont_vars = ["K1","K3","K_change","壹_七_HADS_total","參_四_HADS_total","HADS_change"]
    for v in cont_vars:
        g0 = df[df["group01"]==0][v]
        g1 = df[df["group01"]==1][v]
        print(f"\n=== {v} (Control vs Intervention) ===")
        print(stats.ttest_ind(g0, g1, nan_policy='omit', equal_var=False))

    # ============ McNemar (within-group binary) ============
    binary_pairs = [
        ("壹_五_1.心肺復甦術", "參_二_1.心肺復甦術"),
        ("壹_五_2.氣管內插管", "參_二_2.氣管內插管")
    ]

    for g in [0, 1]:
        print("\n====== Group", g, "McNemar tests ======")
        dfg = df[df["group01"] == g]

        for pre, post in binary_pairs:
            temp = dfg[[pre, post]].dropna()

            table = pd.crosstab(temp[pre], temp[post])
            table = table.reindex(index=[1, 2], columns=[1, 2], fill_value=0)

            print(f"\n{pre} → {post}")
            print("2x2 table (rows=pre, cols=post):")
            print(table)

            result = mcnemar(table, exact=False, correction=True)
            print(f"McNemar statistic = {result.statistic:.4f}, p = {result.pvalue:.4f}")

    # ============ Baseline & post categorical (chi-square) ============
    binary_vars_baseline = [
        "壹_五_1.心肺復甦術",
        "壹_五_2.氣管內插管"
    ]

    binary_vars_post = [
        "參_二_1.心肺復甦術",
        "參_二_2.氣管內插管",
        "參_六_簽署AD意願"
    ]

    def chi_square(df, var):
        ct = pd.crosstab(df[var], df["group01"])
        chi2, p, dof, exp = stats.chi2_contingency(ct)
        return ct, p

    print("\n=== Baseline categorical comparisons ===")
    for v in binary_vars_baseline:
        ct, p = chi_square(df, v)
        print("\n", v, "\n", ct, "\n p =", p)

    print("\n=== Post categorical comparisons ===")
    for v in binary_vars_post:
        ct, p = chi_square(df, v)
        print("\n", v, "\n", ct, "\n p =", p)

    # ============ Logistic regression ============
    df["ADintent01"] = df["參_六_簽署AD意願"].map({1:1,2:0})
    X = sm.add_constant(df[["group01"]])
    y = df["ADintent01"]

    model = sm.Logit(y, X, missing='drop').fit()
    print(model.summary())

    params = model.params
    ci = model.conf_int()
    or_table = pd.DataFrame({
        "OR": np.exp(params),
        "CI_lower": np.exp(ci[0]),
        "CI_upper": np.exp(ci[1])
    })
    print("\nOdds ratios table:")
    print(or_table)

    # ============ K1/K2/K3 within-group ============
    print("\n\n============================================================")
    print("SECTION — WITHIN-GROUP KNOWLEDGE COMPARISON (K1, K2, K3)")
    print("============================================================\n")

    for g in [0, 1]:
        print(f"\nGroup {g} (0 = Control, 1 = Intervention)")
        dfg = df[df["group01"] == g]

        t, p = stats.ttest_rel(dfg["K1"], dfg["K2"], nan_policy="omit")
        print(f"K1 → K2: mean {dfg['K1'].mean():.2f} → {dfg['K2'].mean():.2f}, p = {p:.4f}")

        t, p = stats.ttest_rel(dfg["K2"], dfg["K3"], nan_policy="omit")
        print(f"K2 → K3: mean {dfg['K2'].mean():.2f} → {dfg['K3'].mean():.2f}, p = {p:.4f}")

        t, p = stats.ttest_rel(dfg["K1"], dfg["K3"], nan_policy="omit")
        print(f"K1 → K3: mean {dfg['K1'].mean():.2f} → {dfg['K3'].mean():.2f}, p = {p:.4f}")

    # ============ K1/K2/K3 between-group ============
    print("\n\n============================================================")
    print("SECTION — BETWEEN-GROUP KNOWLEDGE COMPARISON")
    print("============================================================\n")

    for v in ["K1", "K2", "K3"]:
        g0 = df[df.group01 == 0][v]
        g1 = df[df.group01 == 1][v]
        t, p = stats.ttest_ind(g0, g1, equal_var=False, nan_policy="omit")
        print(f"{v}: control mean={g0.mean():.2f}, intervention mean={g1.mean():.2f}, p={p:.4f}")

    df["K2_minus_K1"] = df["K2"] - df["K1"]
    df["K3_minus_K1"] = df["K3"] - df["K1"]
    df["K3_minus_K2"] = df["K3"] - df["K2"]

    for v in ["K2_minus_K1", "K3_minus_K1", "K3_minus_K2"]:
        g0 = df[df.group01 == 0][v]
        g1 = df[df.group01 == 1][v]
        t, p = stats.ttest_ind(g0, g1, equal_var=False, nan_policy="omit")
        print(f"{v}: p = {p:.4f}")


In [4]:
# 1) Show results in notebook
run_full_analysis()

# 2) Save the same results to a text file
import contextlib

output_path = "/content/drive/MyDrive/ACP/ACP_analysis_report_new.txt"
with open(output_path, "w", encoding="utf-8") as f:
    with contextlib.redirect_stdout(f):
        run_full_analysis()

print("\nAnalysis report saved to:", output_path)



========== Group 0 (0=control,1=intervention) ==========

Knowledge Pre vs Post (K1 vs K3):
TtestResult(statistic=np.float64(-0.3637652289278589), pvalue=np.float64(0.7182215582224201), df=np.int64(35))

HADS Pre vs Post:
TtestResult(statistic=np.float64(0.7933902139230844), pvalue=np.float64(0.4335815180061229), df=np.int64(31))

========== Group 1 (0=control,1=intervention) ==========

Knowledge Pre vs Post (K1 vs K3):
TtestResult(statistic=np.float64(0.4814437448655852), pvalue=np.float64(0.6331977202877022), df=np.int64(35))

HADS Pre vs Post:
TtestResult(statistic=np.float64(-0.9208562163316002), pvalue=np.float64(0.36380872699718503), df=np.int64(33))

=== K1 (Control vs Intervention) ===
TtestResult(statistic=np.float64(-0.3797668296450229), pvalue=np.float64(0.7052888701681514), df=np.float64(68.76518550121541))

=== K3 (Control vs Intervention) ===
TtestResult(statistic=np.float64(0.3101720602856713), pvalue=np.float64(0.7573569829140511), df=np.float64(69.600565287684))

===

In [5]:
!pip install pandas statsmodels pingouin xlrd==1.2.0 --quiet

from statsmodels.stats.contingency_tables import mcnemar
import pandas as pd
import numpy as np
from scipy import stats
import statsmodels.api as sm
import pingouin as pg
from google.colab import drive
import re, unicodedata
import io, contextlib

# --------------------------------------------------------
# 0. LOAD DATA & PREP
# --------------------------------------------------------
drive.mount('/content/drive')

file_path = "/content/drive/MyDrive/ACP/ACP_csv1202.csv"

# Use Big5 encoding for Taiwan
df = pd.read_csv(file_path, encoding="big5")

# Clean column names
df.columns = df.columns.str.strip()

# Age: extract year (e.g., "73y8m" → 73)
df["年齡_yr"] = df["年齡"].astype(str).str.extract(r"(\d+)").astype(float)

# Group mapping: 0 = control, 1 = intervention
group_map = {
    "實驗組": 1,
    "對照組": 0,
    1: 1,
    2: 0
}
df["group01"] = df["組別"].map(group_map)

# Knowledge scoring function
def score_knowledge(df, prefix):
    cols = [f"{prefix}_{i}" for i in range(1,7)]
    sub = df[cols]

    corr = pd.DataFrame(index=sub.index)
    corr[cols[0]] = (sub[cols[0]] == 1).astype(int)  # Q1 correct = 1
    for c in cols[1:5]:                             # Q2–Q5 correct = 2
        corr[c] = (sub[c] == 2).astype(int)
    corr[cols[5]] = (sub[cols[5]] == 3).astype(int)  # Q6 correct = 3

    return corr.sum(axis=1)

# K1, K2, K3
df["K1"] = score_knowledge(df, "壹_六_知識問卷")
df["K2"] = score_knowledge(df, "貳_三_知識問卷")
df["K3"] = score_knowledge(df, "參_三_知識問卷")

# Change scores
df["K_change"] = df["K3"] - df["K1"]
df["HADS_change"] = df["參_四_HADS_total"] - df["壹_七_HADS_total"]


# --------------------------------------------------------
# 1. DEFINE ANALYSIS FUNCTION (PRINTS FULL REPORT)
# --------------------------------------------------------
def run_full_analysis():
    # ============================================================
    # SECTION 1 — BASELINE (TABLE 1)
    # ============================================================
    print("============================================================")
    print("SECTION 1 — BASELINE (TABLE 1)")
    print("============================================================\n")

    # Continuous baseline variables
    cont_vars = [
        "年齡_yr", "衰弱程度", "壹_二_MMSE",
        "壹_七_HADS_total", "壹_七_HADS_A", "壹_七_HADS_D", "K1"
    ]

    for v in cont_vars:
        g0 = df[df.group01 == 0][v].dropna()
        g1 = df[df.group01 == 1][v].dropna()
        t, p = stats.ttest_ind(g0, g1, equal_var=False)
        print(f"**{v}**")
        print(f"  Control mean={g0.mean():.2f},  Intervention mean={g1.mean():.2f}")
        print(f"  t-test p={p:.4f}\n")

    # Categorical baseline variables
    cat_vars = [
        "性別","教育程度","宗教信仰","婚姻狀態","收案地點",
        "壹_三_民眾自述健康狀態","壹_四_治療選擇偏好",
        "壹_五_1.心肺復甦術","壹_五_2.氣管內插管"
    ]

    for v in cat_vars:
        print(f"\n-- {v} --")
        ct = pd.crosstab(df[v], df["group01"])
        chi2, p, dof, exp = stats.chi2_contingency(ct)
        print(ct)
        print(f"Chi-square p={p:.4f}")

    # ============================================================
    # SECTION 2 — WITHIN-GROUP PRE–POST (Continuous)
    # ============================================================
    print("\n\n============================================================")
    print("SECTION 2 — WITHIN-GROUP PRE–POST (Continuous)")
    print("============================================================\n")

    for g in [0, 1]:
        print(f"Group {g} (0=Control / 1=Intervention)")
        dfg = df[df["group01"] == g]

        # Knowledge pre→post (K1 vs K3)
        t, p = stats.ttest_rel(dfg["K1"], dfg["K3"], nan_policy='omit')
        print(f"  Knowledge pre→post: p = {p:.4f}")

        # HADS pre→post
        t, p = stats.ttest_rel(dfg["壹_七_HADS_total"], dfg["參_四_HADS_total"], nan_policy='omit')
        print(f"  HADS pre→post: p = {p:.4f}\n")

    # ============================================================
    # SECTION 3 — BETWEEN-GROUP COMPARISONS (Continuous)
    # ============================================================
    print("\n\n============================================================")
    print("SECTION 3 — BETWEEN-GROUP COMPARISONS (Continuous)")
    print("============================================================\n")

    vars_between = [
        "K1", "K3", "K_change",
        "壹_七_HADS_total", "參_四_HADS_total", "HADS_change"
    ]

    for v in vars_between:
        g0 = df[df.group01 == 0][v]
        g1 = df[df.group01 == 1][v]
        t, p = stats.ttest_ind(g0, g1, equal_var=False, nan_policy='omit')
        print(f"{v}: p={p:.4f}")

    # ============================================================
    # SECTION 4 — WITHIN-GROUP PRE–POST (Binary, McNemar)
    # ============================================================
    print("\n\n============================================================")
    print("SECTION 4 — WITHIN-GROUP PRE–POST (Binary, McNemar)")
    print("============================================================\n")

    binary_pairs = [
        ("壹_五_1.心肺復甦術", "參_二_1.心肺復甦術"),
        ("壹_五_2.氣管內插管", "參_二_2.氣管內插管")
    ]

    for g in [0, 1]:
        print(f"\nGroup {g}\n")
        dfg = df[df.group01 == g]

        for pre, post in binary_pairs:
            temp = dfg[[pre, post]].dropna()
            table = pd.crosstab(temp[pre], temp[post])
            table = table.reindex(index=[1,2], columns=[1,2], fill_value=0)

            print(f"{pre} → {post}")
            print(table)
            result = mcnemar(table, exact=False, correction=True)
            print(f"  McNemar p={result.pvalue:.4f}\n")

    # ============================================================
    # SECTION 5 — BETWEEN-GROUP (Binary)
    # ============================================================
    print("\n\n============================================================")
    print("SECTION 5 — BETWEEN-GROUP (Binary)")
    print("============================================================\n")

    binary_post = [
        "參_二_1.心肺復甦術",
        "參_二_2.氣管內插管",
        "參_六_簽署AD意願"
    ]

    for v in binary_post:
        ct = pd.crosstab(df[v], df["group01"])
        chi2, p, dof, exp = stats.chi2_contingency(ct)
        print(f"\n{v}")
        print(ct)
        print(f"Chi-square p={p:.4f}")

    # ============================================================
    # SECTION 6 — MAIN OUTCOME LOGISTIC REGRESSION
    # ============================================================
    print("\n\n============================================================")
    print("SECTION 6 — MAIN OUTCOME LOGISTIC REGRESSION")
    print("============================================================\n")

    df["ADintent01"] = df["參_六_簽署AD意願"].map({1:1, 2:0})
    X = sm.add_constant(df[["group01"]])
    y = df["ADintent01"]

    logit_model = sm.Logit(y, X, missing='drop').fit()
    print(logit_model.summary())

    print("\nOdds ratios:")
    params = logit_model.params
    conf = logit_model.conf_int()
    or_table = pd.DataFrame({
        "OR": np.exp(params),
        "CI_low": np.exp(conf[0]),
        "CI_high": np.exp(conf[1])
    })
    print(or_table)

    # ============================================================
    # SECTION 7 — WITHIN-GROUP KNOWLEDGE (K1, K2, K3)
    # ============================================================
    print("\n\n============================================================")
    print("SECTION 7 — WITHIN-GROUP KNOWLEDGE (K1, K2, K3)")
    print("============================================================\n")

    for g in [0, 1]:
        dfg = df[df["group01"] == g]
        print(f"\nGroup {g} (0=Control / 1=Intervention)")
        print(f"  K1 mean = {dfg['K1'].mean():.2f}")
        print(f"  K2 mean = {dfg['K2'].mean():.2f}")
        print(f"  K3 mean = {dfg['K3'].mean():.2f}")

        # K1 → K2
        t, p = stats.ttest_rel(dfg["K1"], dfg["K2"], nan_policy='omit')
        print(f"  K1 → K2: p = {p:.4f}")

        # K2 → K3
        t, p = stats.ttest_rel(dfg["K2"], dfg["K3"], nan_policy='omit')
        print(f"  K2 → K3: p = {p:.4f}")

        # K1 → K3
        t, p = stats.ttest_rel(dfg["K1"], dfg["K3"], nan_policy='omit')
        print(f"  K1 → K3: p = {p:.4f}")

    # ============================================================
    # SECTION 8 — BETWEEN-GROUP KNOWLEDGE (K1, K2, K3 & changes)
    # ============================================================
    print("\n\n============================================================")
    print("SECTION 8 — BETWEEN-GROUP KNOWLEDGE (K1, K2, K3 & Changes)")
    print("============================================================\n")

    # timepoint comparisons
    for v in ["K1", "K2", "K3"]:
        g0 = df[df.group01 == 0][v]
        g1 = df[df.group01 == 1][v]
        t, p = stats.ttest_ind(g0, g1, equal_var=False, nan_policy='omit')
        print(f"{v}: control mean={g0.mean():.2f}, intervention mean={g1.mean():.2f}, p={p:.4f}")

    # change scores
    df["K2_minus_K1"] = df["K2"] - df["K1"]
    df["K3_minus_K1"] = df["K3"] - df["K1"]
    df["K3_minus_K2"] = df["K3"] - df["K2"]

    for v in ["K2_minus_K1", "K3_minus_K1", "K3_minus_K2"]:
        g0 = df[df.group01 == 0][v]
        g1 = df[df.group01 == 1][v]
        t, p = stats.ttest_ind(g0, g1, equal_var=False, nan_policy='omit')
        print(f"{v}: p={p:.4f}")


# --------------------------------------------------------
# 2. RUN ONCE, CAPTURE, PRINT, SAVE TO FILE
# --------------------------------------------------------
buf = io.StringIO()
with contextlib.redirect_stdout(buf):
    run_full_analysis()

report_text = buf.getvalue()

# print to notebook cell
print(report_text)

# save to txt file
output_path = "/content/drive/MyDrive/ACP/ACP_analysis_report_with_K123.txt"
with open(output_path, "w", encoding="utf-8") as f:
    f.write(report_text)

print("\nReport saved to:", output_path)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 4.0 MB/s eta 0:00:00
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
SECTION 1 — BASELINE (TABLE 1)

**年齡_yr**
  Control mean=80.25,  Intervention mean=80.19
  t-test p=0.9746

**衰弱程度**
  Control mean=4.92,  Intervention mean=4.81
  t-test p=0.5725

**壹_二_MMSE**
  Control mean=25.47,  Intervention mean=24.58
  t-test p=0.3590

**壹_七_HADS_total**
  Control mean=8.19,  Intervention mean=8.83
  t-test p=0.6251

**壹_七_HADS_A**
  Control mean=2.39,  Intervention mean=2.19
  t-test p=0.7533

**壹_七_HADS_D**
  Control mean=5.81,  Intervention mean=6.64
  t-test p=0.4252

**K1**
  Control mean=2.97,  Intervention mean=3.11
  t-test p=0.7053


-- 性別 --
group01   0   1
性別             
1        16  20
2        20  16
Chi-square p=0.4795

-- 教育程度 --
group01   0   1
教育程度           
1         3   0
2         1   2
3        12   9
4         4   6
5         6  10
6